# FUTURE
### Fourier Transformation Tool to Unravel Renewable Intermittency in the European Power System of the Future

__Welcome!__ This Notebook is split in three sections:
1. Building the database
    * Importing the data, <br>
    * Applying a capacity scenario,<br>
    * Adding custom regions,<br>
    * Adding custom variables (e.g. residual load)
2. Flexibility requirements analysis with Fourier decomposed time-series
    * Decomposed time series visualization
    * Flexibility requirements visualization
    * Energy yield visualization
3. Interrregional correlation analysis between different variables and spectra

In [ ]:
# First, let's set up the environment

import pandas as pd
from src.pecd_handling import read_pecd_xls_file, add_custom_variable, add_custom_region
from src.helpers.fft import add_decomposed_ts

%load_ext autoreload
%autoreload 2

import plotly.io as pio
pio.templates.default = "seaborn"

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))


The following Javascript line is required so that the plots are not shown in a scrollable window but as collapsed windows

In [ ]:
%%javascript 
IPython.OutputArea.prototype._should_scroll = function(lines) {return false;}

## Building the database

We have added a small sample database in the repository natively. In order to work with the full PECD dataset, please refer to the links in the README and download the respective excel files.

In [ ]:
# --- LOADING RES AND LOAD TIME SERIES FROM THE PECD FORMAT --- #

pv =       read_pecd_xls_file('.\\data\\pecd_sample\\PECD_2030_PV.xlsx')
onshore =  read_pecd_xls_file('.\\data\\pecd_sample\\PECD_2030_Onshore.xlsx')
offshore = read_pecd_xls_file('.\\data\\pecd_sample\\PECD_2030_Offshore.xlsx')
load =     read_pecd_xls_file('.\\data\\pecd_sample\\MAF2020_DemandTimeSeries_2030.xlsx')

df = pd.concat({'load': load, 'pv': pv, 'onshore': onshore, 'offshore': offshore}, names=['variable'], axis=1)
df = df.reorder_levels(['region', 'variable'], axis=1)
del pv, onshore, offshore, load  # no need for those anymore, they're in df now.

In [ ]:
# --- LOADING CAPACITY SCENARIO AND SCALING RES TIME SERIES --- #

# You can choose an available sheetname from the RES_capacity_scenarios Excel file
caps = pd.read_excel('data/RES_capacity_scenarios.xlsx', sheet_name='MAF_2030', index_col=0)
caps.columns.name = 'variable'
caps.loc[:, 'load'] = 1  # 'load' must be set here as a scaling factor for the next setp

df = df.multiply(caps.stack(), axis=1).dropna(axis=1)  # scaling the RES time series

In [ ]:
# You can add custom regions if you wish to understand a hypothetical merge between those
df = add_custom_region(df, 'custom_region_1', ['DE00', 'BE00', 'FR00'], method='sum')

In [ ]:
# If you have a subset of regions that is of interest, make the dataframe smaller and as a result your operations will be faster
# df = df[['DE00', 'BE00', 'FR00', 'custom_region_1']]

In [ ]:
# Custom variables can be defined by defining the sum of existing variables with respective multipliers
df = add_custom_variable(df, 'RES_sum', {'offshore': 1, 'onshore': 1, 'pv': 1})  # the sum of all three RES technologies
df = add_custom_variable(df, 'residual_load', {'load': 1, 'offshore': -1, 'onshore': -1, 'pv': -1})
df = df.sort_index(axis=1)

In [ ]:
df.head().round(2)

### Adding Fourier Decomposition
In the following step we will add an index level with the fourier decomposed time series for different spectra.

You can set a boolean parameter for removing the DC componenet as well as accumulating the spectra.

__remove_dc:__ If you remove the DC component, you will remove the mean from the time series. The decomposed Fourier spectrum will thus be centered around zero. In the paper, the DC component was indeed removed.

__accumulate_spectra:__ If you accumulate spectra, the spectra are not strictly split. Rather, each spectrum includes the sum of itself plus all spectrums above (e.g. the mongthly spectrum is actually the sum of the seasonal + monthly, and so on). This is useful especially for the flexibility requirement analysis, as it enables you to identify the flexibility requirements for the sum of the spectra.

In [ ]:
df = add_decomposed_ts(df, remove_dc=True, accumulate_spectra=True)

In [ ]:
# feel free to have a look at the DataFrame structure with the various spectra
df.head().round(2)

## Dashboards
### Time Series Decomposition

In [ ]:
from src.dashboards import TimeSeriesDashboard
tsd = TimeSeriesDashboard(df)
tsd.interact()

# In the UI, please select what you want to visualize for the decomposed time series analysis.
# In the first row you can select a subset of the data that you want to visualize (e.g. regions, variables, spectra)
# In the second row you can setup the plot grid. 
# Every level (regions, variables, spectra) for which you have selected more than one item,
# should definitely be included in the plot grid setup (rows, columns, color, line dash).
# In addition, you can enable to show the (95% / 5%) confidence interval as line_dash option.

### Flexibility Requirements Analysis 
 
TODO: Comment on upward flexibility requirements

In [ ]:
from src.dashboards import FlexibilityRequirementDashboard
frd = FlexibilityRequirementDashboard(df)
frd.interact()

 ### Energy Yield Analysis 

In [ ]:
from src.dashboards import EnergyYieldDashboard
eyd = EnergyYieldDashboard(df)
eyd.interact()

# The values in TWh are annualized

### Interrregional correlation analysis between different variables and spectra